In [1]:
import scipy as sp
import matplotlib as plt
import jax
import jax.numpy as jnp
import os
import pandas as pd
from ast import literal_eval
import sys
import time
from IPython.display import clear_output

#Redirect print output.
f = open('NRFullResults.txt', 'w')
sys.stdout = f

#Method Based On:
#Dimitrakopoulos, P., Jia, W. & Li, C. 
#An Improved Computational Method for the Calculation of Mixture Liquid–Vapor Critical Points. 
#Int J Thermophys 35, 865–889 (2014). https://doi.org/10.1007/s10765-014-1680-7

In [2]:
def LookUpMix(MxN):
    ts = time.time()
    #Read from mixture.csv file to get components.
    path = os.getcwd()
    Mdf = pd.read_csv(path + "/Mixture Compositions.csv", index_col = "Mixture No.")
    mixture = Mdf.iloc[MxN, :]
    mxNames = mixture.index[mixture.notnull()]  
    
    #Assign compositions vlaues for components.
    y = mixture[mixture.notnull()].to_numpy()
    
    
    #Define global variables from mixture props.
    R = 8.31446 #J/(mol*K)
    
    #Obtain component critical values from critical properties list.
    Cdf = pd.read_csv(path + "/Chemical Critical Properties.csv")
    Pc = Cdf.loc[Cdf["Component"].isin(mxNames), "Pc (MPa)"].to_numpy()*10**6 # Pa
    Tc = Cdf.loc[Cdf["Component"].isin(mxNames), "Tc (K)"].to_numpy() #K
    Vc = Cdf.loc[Cdf["Component"].isin(mxNames), "Vc (l/mol)"].to_numpy()*0.001 #m^3
    w = Cdf.loc[Cdf["Component"].isin(mxNames), "w"].to_numpy()
    
    #Define mol vector and number of components.
    n_tot = 1 #mol
    n = y*n_tot
    C = len(n)
    
    #Unpack binaqry interaction coefficients
    kdf = pd.read_csv(path + "/Binary Interactions.csv", index_col = "Component")
    k = []
    for i in range(C):
        klist = kdf.loc[mxNames[i], :]
        klist = klist.loc[klist.index.isin(mxNames)]
        k.append(klist.to_numpy())
    k = jnp.array(k)

    EOS = 'SRK'
    print("Components: "+str(mxNames.values)+" At:"+str(y)+" Using "+EOS)
    print("---------------------------------------------")
    display(time.time()-ts)
    return y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k

In [3]:
#N_bar, the total change in mols.
def N_bar(dnvec):
    return jnp.sum((dnvec))

In [4]:
#nu, a constant dependant on EOS, that affects a.
def nu(EOS):
    if EOS == 'SRK':
        return 0.42748
    if EOS == 'PR':
        return 0.45724

In [5]:
#c, the accentricity polynomial.
def c(i, EOS, w):
    if EOS == 'SRK':
        ci = 0.48
        ci += 1.574*w[i]
        ci += -0.176*w[i]**2
        return ci
    if EOS == 'PR':
        ci = 0.37464
        ci += 1.54226*w[i]
        ci += -0.26992*w[i]**2
        return ci

In [6]:
#ai, the attraction paramater of component i.
def ai(i, Tc_mx, Tc, R, EOS, Pc, w):
    ai = (R*Tc[i])**2*nu(EOS)/Pc[i]
    ai = ai*(1 + c(i, EOS, w)*(1-(Tc_mx/Tc[i])**0.5))**2
    return ai

In [7]:
#aij, the binary attraction paramater of component system i-j.
def aijf(Tc_mx, Tc, R, EOS, Pc, w, k, C):
    aij = jnp.zeros([C, C])
    for i in range(C):
        for j in range(C):
            aij = aij.at[i,j].set((ai(i, Tc_mx, Tc, R, EOS, Pc, w)*ai(j, Tc_mx, Tc, R, EOS, Pc, w))**0.5*(1-k[i][j]))
    return aij

In [8]:
#bi, the repulsion paramater of component i.
def b(i, EOS, R, Tc, Pc):
    if EOS == 'SRK':
        bi = 0.08664*R*Tc[i]/Pc[i]
        return bi
    if EOS == 'PR':
        bi = 0.07780*R*Tc[i]/Pc[i]
        return bi

In [9]:
#D1, a parameter that defines the EOS.
def D1(EOS):
    if EOS == 'SRK':
        u0 = 1
        w0 = 0
    if EOS == 'PR':
        u0 = 2
        w0 = -1      
    D1 = (u0 + jnp.sqrt(u0**2-4*w0))/2
    return D1

In [10]:
#D2, a parameter that defines the EOS.
def D2(EOS):
    if EOS == 'SRK':
        u0 = 1
        w0 = 0
    if EOS == 'PR':
        u0 = 2
        w0 = -1      
    D2 = (u0 - jnp.sqrt(u0**2-4*w0))/2
    return D2

In [11]:
#a_tot, the weighted sum of binary attraction interactions.
def a_tot(n, n_tot, aij, C):
    a_t = 0
    for i in range(C):
        for j in range(C):
            a_t += n[i]*n[j]/n_tot**2*aij[i, j]
    return a_t

In [12]:
#b_tot, the weighted sum of repulsion interactions.
def b_tot(y, EOS, R, Tc, Pc, C):
    b_t = 0
    for i in range(C):
        b_t += y[i]*b(i, EOS, R, Tc, Pc)
    return b_t

In [13]:
#alphak, the relative attraction of all interactions with component k.
def alpha(j, y, aij, n, n_tot, C):
    alpk = 0
    for i in range(C):
        alpk += y[i]*aij[i,j]
    return alpk/a_tot(n, n_tot, aij, C)

In [14]:
#alpha_bar, total change in the alphaks from delta_n.
def alpha_bar(dnvec, y, aij, n, n_tot, C):
    alp_bar = 0
    for i in range(C):
        alp_bar += dnvec[i]*alpha(i, y, aij, n, n_tot, C)
    return alp_bar

In [15]:
#a_bar, the realtive change in a_tot due from delta_n.
def a_bar(dnvec, aij, n, n_tot, C):
    a_b = 0
    for i in range(C):
        for j in range(C):
            a_b += dnvec[i]*dnvec[j]*aij[i,j]
    a_b = a_b/a_tot(n, n_tot, aij, C)
    return a_b

In [16]:
#betai, the relative repulsion of component k.
def beta(i, EOS, R, Tc, Pc, y, C):
    return b(i, EOS, R, Tc, Pc)/b_tot(y, EOS, R, Tc, Pc, C)

In [17]:
#beta_bar, the total change in betais from delta_n.
def beta_bar(dnvec, EOS, R, Tc, Pc, y, C):
    bet_bar = 0
    for i in range(C):
        bet_bar += dnvec[i]*beta(i, EOS, R, Tc, Pc, y, C)
    return bet_bar

In [18]:
#K, the ratio of critcal volume of mixture to repulsion parameter.
def K(Vc_mx, y, EOS, R, Tc, Pc, C):
    return Vc_mx/b_tot(y, EOS, R, Tc, Pc, C)

In [19]:
#F1-F6 are EOS based factors which are f(D1, D2, K)
def F1(Kv, EOS):
    return 1/(Kv-1)

def F2(Kv, EOS):
    F2 = 2/(D1(EOS)-D2(EOS))
    F2 *= (D1(EOS)/(Kv+D1(EOS))-D2(EOS)/(Kv+D2(EOS)))
    return F2

def F3(Kv, EOS):
    F3 = 1/(D1(EOS)-D2(EOS))
    F3 *= ((D1(EOS)/(Kv+D1(EOS)))**2-(D2(EOS)/(Kv+D2(EOS)))**2)
    return F3

def F4(Kv, EOS):
    F4 = 1/(D1(EOS)-D2(EOS))
    F4 *= ((D1(EOS)/(Kv+D1(EOS)))**3-(D2(EOS)/(Kv+D2(EOS)))**3)
    return F4

def F5(Kv, EOS):
    F5 = 2/(D1(EOS)-D2(EOS))
    F5 *= jnp.log((Kv+D1(EOS))/(Kv+D2(EOS)))
    return F5

def F6(Kv, EOS):
    F6 = 2/(D1(EOS)-D2(EOS))
    F6 *= (D1(EOS)/(Kv+D1(EOS))-D2(EOS)/(Kv+D2(EOS)))-jnp.log((Kv+D1(EOS))/(Kv+D2(EOS)))
    return F6

In [20]:
#Generalized vector of functions that descirbes a cubic equation of state.
from functools import partial
@partial(jax.jit, static_argnames=['EOS', 'C', 'n_tot', 'R'])
def GeneralizedCubicFunction(xvec, y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k):
    #Unpack input xvec.
    Tc_mx = xvec[0]
    Vc_mx = xvec[1]
    dnvec = xvec[2:]

    fvec = jnp.zeros([1,len(xvec)])
    
    #Calculate variable values
    aij = aijf(Tc_mx, Tc, R, EOS, Pc, w, k, C)
    a_totv = a_tot(n, n_tot, aij, C)
    b_totv = b_tot(y, EOS, R, Tc, Pc, C)
    N_barv = N_bar(dnvec)
    a_barv = a_bar(dnvec, aij, n, n_tot, C)
    alpha_barv = alpha_bar(dnvec, y, aij, n, n_tot, C)
    beta_barv = beta_bar(dnvec, EOS, R, Tc, Pc, y, C)
    
    Kv = K(Vc_mx, y, EOS, R, Tc, Pc, C)
    
    F1v = F1(Kv, EOS)
    F2v = F2(Kv, EOS)
    F3v = F3(Kv, EOS)
    F4v = F4(Kv, EOS)
    F5v = F5(Kv, EOS)
    F6v = F6(Kv, EOS)
    
    
    #First derivative of each component's Helmholtz Energy should be 0. 
    for i in range(C):
        betav = beta(i, EOS, R, Tc, Pc, y, C)
        pA = R*Tc_mx/n_tot
        pB = a_totv/(b_totv*n_tot)
        A1 = dnvec[i]/y[i]
        A2 = F1v*(betav*N_barv + beta_barv)
        A3 = betav*F1v**2*beta_barv
        B1 = betav*beta_barv*F3v
        B2 = 0
        for j in range(C):
            B2 += dnvec[j]*aij[i,j]
        B2 = -F5v/a_totv*B2
        B3 = F6v*(betav*beta_barv-alpha(i, y, aij, n, n_tot, C)*beta_barv-alpha_barv*betav)
        A = pA*(A1+A2+A3)
        B = pB*(B1+B2+B3)
        fvec = fvec.at[0,i].set(A+B)
    
    #Sum of all second derivatives of Helmholtz Energy should be 0.
    pA = R*Tc_mx/n_tot**2
    pB = a_totv/(b_totv*n_tot**2)
    A1 = 0
    for i in range(C):
        A1 += dnvec[i]**3/y[i]**2
    A1 = -A1
    A2 = 3*(N_barv*(beta_barv*F1v)**2)
    A3 = 2*((F1v*beta_barv)**3)
    B1 = 3*(beta_barv**2)*((2*alpha_barv-beta_barv)*(F3v+F6v))
    B2 = -2*(beta_barv**3)*F4v
    B3 = -3*beta_barv*a_barv*F6v
    A = pA*(A1+A2+A3)
    B = pB*(B1+B2+B3)
    fvec = fvec.at[0,C].set(A+B)
    
    #Euclidean distance of delta_n should be 1.
    norm_mols = 0
    for i in range(C):
        norm_mols += dnvec[i]**2
    norm_mols += -1
    fvec = fvec.at[0,C+1].set(norm_mols)
    return fvec

In [21]:
def InitializeNR(y, Vc, Tc, C, n):
    #Initialize first guess based on composition and component critical values.
    dn0 = jnp.zeros(jnp.shape(n))
    Tc0 = 0
    Vc0 = 0
    for i in range(C):
        Vc0 += y[i]*Vc[i]
        Tc0 += y[i]*Tc[i]
        dn0 = dn0.at[i].set(y[i]**(2/3))
    Tc0 = 3*Tc0
    return Tc0, Vc0, dn0

In [22]:
def NewtonRaphson(MxN):
    #LookUpMixture to create all nesseccary globals.
    [y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k] = LookUpMix(MxN)
    
    #Initialize first xvec.
    temptc, tempvc, tempdn = InitializeNR(y, Vc, Tc, C, n)
    x0vec = jnp.array([temptc, tempvc])
    x0vec = jnp.append(x0vec, tempdn)
    print("Initial X-Vector Guess:")
    print(x0vec)
    print("---------------------------------------------")
    
    start_time = time.time()
    
    #Create Jacobian matrix function and initialize F matrix.
    Fmat = jnp.zeros([1, len(x0vec)])
    #GCF = jax.jit(GeneralizedCubicFunction)
    #Jmat = jax.jacfwd(GCF)
    Jmat = jax.jacfwd(GeneralizedCubicFunction)
    
    #Iterate with x(k+1) = x(k) + D*dx.
    xvec = x0vec
    dxvec = jnp.ones(jnp.shape(xvec))
    itr = 0
    itr_time = 0
    
    print("dxVector Magnitudes:")
    while (abs(dxvec[0])>1e-4 or abs(dxvec[1])>10e-8 or any(abs(dxvec[2:])>1e-4)):
        #Display to console current run status.
        clear_output(wait = True)
        display("Current Status:")
        display("Mixture "+str(MxN)+" Iteration "+str(itr))
        display("Last Iteration Time: "+str(itr_time))
        
        #Count iterations for dampening factor.
        itr_start = time.time()
        itr += 1
        
        #Calcualte function values and Jacobian at xvec(k).
        F = GeneralizedCubicFunction(xvec, y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k)
        cubtime = time.time()
        Fmat = jnp.append(Fmat, F, axis = 0)
        F = -1*jnp.transpose(F)
        J = Jmat(xvec,y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k)[0]

        #Solve dxvec, uses LU decomposition with partial pivoting.
        dxvec = jnp.linalg.solve(J, F)
        dxvec = jnp.transpose(dxvec)
        dxvec = jnp.reshape(dxvec, [jnp.shape(J)[0]])
        
        #Define Q, the dampening factor. 0 for binary mixtures, and 518 for ~13 iterations for non-binary.
        if C ==2:
            Q = 0
        else:
            Q =518
        #Apply dampening.
        D = 1/(1+Q*jnp.exp(-0.5*itr))
        xvec = xvec + D*dxvec

        #Max iterations is set as 30.
        if itr>30:
            print("Convergence not achieved in 30 iterations.")
            Pc = None
            return Fmat, xvec, Pc
        
        #Calculate iteration time for full report.
        itr_end = time.time()
        itr_time = itr_end-itr_start
        print(str(itr)+"    "+str(round(jnp.linalg.norm(dxvec), 4))+"    "+str(round((itr_time), 4)))
        
    #Calculate Mixture runtime for summary report.    
    end_time = time.time()
    rn_time = end_time-start_time
    
    #Calculate final function values for evaltuation of convergence.
    F = GeneralizedCubicFunction(xvec, y, Tc, Pc, w, EOS, C, n, n_tot, R, Vc, k)      
    Fmat = jnp.append(Fmat[1:, :], F, axis = 0)
    
    #Check convergence and calculate Pc.
    conv_flag = jnp.linalg.norm(F-jnp.zeros(jnp.shape(F)))
    if conv_flag >= 1:
        print("Convergence achieved, function values high. Critical point may be false or nonexistant.")
    else:
        print("Magnitude of final function vector: " + str(round(conv_flag, 4)))
    
    print("---------------------------------------------")
    aij = aijf(xvec[0], Tc, R, EOS, Pc, w, k, C)
    Pc = R*xvec[0]/(xvec[1]-b_tot(y, EOS, R, Tc, Pc, C)) - a_tot(n, n_tot, aij, C)/((xvec[1]+D1(EOS)*b_tot(y, EOS, R, Tc, Pc, C))*(xvec[1]+D2(EOS)*b_tot(y, EOS, R, Tc, Pc, C)))
    
    return Fmat, xvec, Pc, itr, rn_time

In [23]:
def RunMixDataSet():
    path = os.getcwd()
    Mdf = pd.read_csv(path + "/Mixture Compositions.csv", index_col = "Mixture No.")
    pd.set_option("display.precision", 5)
    dfRes = pd.DataFrame(index = Mdf.index, columns = ["Pc (MPa)", "Tc (K)", "Vc (L/mol)", "Rhoc (mol/L)", "NumIter", "Time (s)"])
    
    for MxN in range(jnp.shape(Mdf)[0]):
        
        Fmat, xvec, Pc, itr, rn_time = NewtonRaphson(MxN)

        print("Magnitudes of function vector at each iteration.")
        print(jnp.linalg.norm(Fmat, axis = 1))
        print("---------------------------------------------")

        print("Critical Pressure: " + str(Pc/10**6) + " MPa")
        print("Critical Temperature: " + str(xvec[0]) + " K")
        print("Critical Density: " + str(1/xvec[1]) + " mol/m^3")
        print("Delta Composition: " + str(xvec[2:])+ " mol")
        print("********************************************")
        print()
        
        dfRes.iloc[MxN, :] = [Pc/10**6, xvec[0], xvec[1]/0.001, 0.001/xvec[1], itr, rn_time]
    
    fSum = open("NRResultsSummary.txt", "w")
    fSum.write(dfRes.to_csv())
    fSum.close()

    return dfRes

In [24]:
dfRes = RunMixDataSet()

'Current Status:'

'Mixture 43 Iteration 19'

'Last Iteration Time: 0.004018306732177734'

In [25]:
dfRes

,Pc (MPa),Tc (K),Vc (L/mol),Rhoc (mol/L),NumIter,Time (s)
Mixture No.,,,,,,
1,5.31732,299.3391,0.16383211,6.1038094,7,3.32614
2,8.071614,300.71432,0.10928671,9.150244,7,0.06028
3,6.351146,441.5949,0.2599774,3.8464882,20,5.6449
4,7.55824,390.7197,0.1904014,5.252062,20,0.16947
5,6.268997,405.25946,0.22510554,4.4423604,20,0.16043
6,8.38276,395.76572,0.18522663,5.398792,20,0.23127
7,7.063187,423.0236,0.23113193,4.3265333,20,0.15364
8,6.924063,416.99686,0.22792186,4.3874683,20,0.16121
9,9.135976,323.33612,0.14568108,6.8643093,20,0.15999


In [26]:
dfRes.sum()

Pc (MPa)              nan
Tc (K)                nan
Vc (L/mol)            nan
Rhoc (mol/L)          nan
NumIter               856
Time (s)        190.85593
dtype: object